## Introduction:


Concrete ML (CML) now supports model compilation to the TFHE-rs format, enabling deeper interoperability between the CML and TFHE-rs frameworks. This allows developers to combine CML’s intuitive, Python-based workflow with the performance and low-level control of TFHE-rs for advanced privacy-preserving applications.

This notebook showcases a complete binary classification use case. We first perform encrypted inference using a CML decision tree model. The resulting ciphertexts are then exported to a TFHE-rs program for additional post-processing. In our example, this post-processing includes applying an `argmax` operation on the model's encrypted output, followed by a multiplication with a random plaintext token. If the predicted class is 0, the program returns a vector of zeros; otherwise, it returns the random token.

This pattern represents a typical privacy-preserving server-side authentication flow, where access to a secure token is granted only if the encrypted model prediction satisfies a certain condition — without ever decrypting sensitive data throughout the process.

In [1]:
import subprocess
from tempfile import TemporaryDirectory

import concrete_ml_extensions as fhext
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

from concrete.ml.common.utils import CiphertextFormat
from concrete.ml.deployment import FHEModelClient, FHEModelDev, FHEModelServer
from concrete.ml.sklearn import DecisionTreeClassifier as ConcreteDecisionTreeClassifier

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


In [2]:
# Generate toy classification dataset
x, y = make_classification(n_samples=100, class_sep=2, n_features=10, random_state=42)

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [3]:
# For now, only 8-bit quantization is supported for TFHE-rs interoperability
model = ConcreteDecisionTreeClassifier(n_bits=8)
# Train the CML model on clear data
model.fit(X_train, y_train)

DecisionTreeClassifier(n_bits={'op_inputs': 8, 'op_leaves': 8})

In [4]:
# Compile the model with TFHE-rs support (ciphertext_format=CiphertextFormat.TFHE_RS)
model.compile(X_train, ciphertext_format=CiphertextFormat.TFHE_RS);

In [5]:
# Use a temporary directory to store the Dev, client, and server models
with TemporaryDirectory() as temp_dir:
    # Export the model for deployment using the FHEModelDev API
    model_dev = FHEModelDev(temp_dir, model)
    model_dev.save()

    # Loading the model on the client side using FHEModelClient API
    fhe_model_client = FHEModelClient(path_dir=temp_dir)
    fhe_model_client.load()

    # Loading the model on the server side using FHEModelServer API
    fhe_model_server = FHEModelServer(temp_dir)
    fhe_model_server.load()

In [6]:
# Client side : Generates all keys and serializes the evaluation keys required for the server
evaluation_keys, tfhers_evaluation_keys = fhe_model_client.get_serialized_evaluation_keys()

with open("evalkeys_tfhers.bin", "wb") as fp:
    fp.write(tfhers_evaluation_keys)

### Use Case: Privacy-Preserving Authentication with Homomorphic Encryption

In this scenario, the client attempts to authenticate using sensitive data (e.g., a biometric vector such as facial recognition). On the client side, the input is first quantized (to meet FHE constraints), then encrypted, and finally sent to the server.

The server applies a binary classification algorithm — in this case, a decision tree — directly on the encrypted data to determine whether authentication should succeed (decision = 1) or fail (decision = 0). The decision result is then multiplied (homomorphically) by a secret authentication token known only to the server. The encrypted result is sent back to the client.

Upon decryption, the client receives:

the original token if the decision was 1 (authentication successful), or

a vector of zeros if the decision was 0 (authentication failed).

**Disclaimer**: This use case assumes an honest-but-curious server and does not constitute a complete secure authentication protocol. Additional protocol design is required to protect against malicious server scenarios

In [7]:
for DATA_INDEX in range(10):

    print(f"🔢 Test sample #{DATA_INDEX}", "=" * 50)
    print("Inference input:", X_test[[DATA_INDEX]])
    print(f"🎯 Ground truth label: {y_test[DATA_INDEX]}")

    # Client side : Quantize, encrypt and serialize the data
    q_x_encrypted_serialized = fhe_model_client.quantize_encrypt_serialize(X_test[[DATA_INDEX]])

    # Server side: Run the model over encrypted data
    q_y_pred_encrypted_serialized = fhe_model_server.run(q_x_encrypted_serialized, evaluation_keys)

    # Client side : Decrypt, de-quantize and post-process the result
    y_pred = fhe_model_client.deserialize_decrypt_dequantize(q_y_pred_encrypted_serialized[0])

    print("Model logits (after decryption):", y_pred.reshape((-1,)))

    with open("prediction_non_preprocessed.bin", "wb") as f:
        f.write(q_y_pred_encrypted_serialized[0])

    # ***************************************
    # NOW RUN cargo run --release
    # **************************************

    result = subprocess.run(
        [
            "cargo",
            "run",
            "--release",
            "--",  # Required to pass args to the Rust binary
            "evalkeys_tfhers.bin",
            "prediction_non_preprocessed.bin",
            "tfhers_sign_result.bin",
        ],
        capture_output=True,
        text=True,
        check=True,
    )

    with open("tfhers_sign_result.bin", "rb") as f:
        tfhers_postproc_bytes = f.read()

    results = fhext.decrypt_radix(
        blob=tfhers_postproc_bytes,
        shape=(1, 16),
        bitwidth=8,
        is_signed=True,
        secret_key=fhe_model_client.get_tfhers_secret_key(),
    )

    token = results.reshape((-1,))

    if all(t == 0 for t in token):
        print(f"🔐 Authentication token: 🛑 Access denied — token: {token}\n")
    else:
        print(f"🔐 Authentication token: 🟢 Access granted — token: {token} \n")

🔢 Test sample #0 ==================================================
Inference input: [[-0.07015438  2.85605469 -1.05319823  2.4898248  -0.21910053  0.32692737
  -2.21113531  0.77086519  0.82940558  0.23561456]]
🎯 Ground truth label: 0
Model logits (after decryption): [1. 0.]
🔐 Authentication token: 🛑 Access denied — token: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]

🔢 Test sample #1 ==================================================
Inference input: [[-2.43430912  2.25291095  2.09885263  1.23502851 -0.24751864 -0.72713718
   0.6206721  -1.33534436 -0.07443343  0.177701  ]]
🎯 Ground truth label: 1
Model logits (after decryption): [0. 1.]
🔐 Authentication token: 🟢 Access granted — token: [ -50  101   99   77  -62 -122 -112 -100  -60   24 -125   44   93 -103
  -87  -23] 

🔢 Test sample #2 ==================================================
Inference input: [[-3.37679987  3.30450019  2.83991037  1.87087893 -0.37482081 -0.2403254
   0.44426331  1.1593298   0.71095997 -0.36096617]]
🎯 Ground truth labe

## Conclusion

In this notebook, we showed how to combine Concrete ML (CML) and TFHE-rs for secure, end-to-end encrypted processing. After training and compiling a CML decision tree, we performed an encrypted inference and then applied additional post-processing in TFHE-rs, all while preserving data privacy.

This approach highlights the power of interoperability between CML and TFHE-rs, and paves the way for more advanced privacy-preserving use cases.